## Машинное обучение, ВМК МГУ

## Практическое задание 2

### Общая информация
Дата выдачи: 9 октября 2019

Максимальная оценка: 10 баллов + 1 бонусный балл

Мягкий дедлайн: 23:59MSK 23 октября (за каждый день просрочки снимается 1 балл)

Жесткий дедлайн: 23:59MSK 30 октября.

### О задании

В этом задании вы:
- Познакомитесь с методом решения задачи регрессии на основе метода ближайших соседей.
- Реализуете алгоритм kNN для задачи регрессии.
- Изучите методы работы с категориальными и текстовыми переменными.


### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.


### Формат сдачи
Для сдачи задания переименуйте получившийся файл *.ipynb в соответствии со следующим форматом: homework-practice-02-Username.ipynb, где Username — ваша фамилия и имя на латинице именно в таком порядке (например, homework-practice-02-ivanov.ipynb).

Далее отправьте этот файл на anytask в соответсвующий раздел.

In [1]:
import numpy as np
import pandas as pd

Все эксперименты в этой лабораторной работе предлагается проводить на данных соревнования New York City Airbnb Open Data: https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data#AB_NYC_2019.csv

В данной задаче предлагается предсказать цену на съем квартиры в зависимости от её параметров.

In [2]:
data = pd.read_csv('AB_NYC_2019.csv')
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [3]:
data.shape

(48895, 16)

In [4]:
# число значений у признаков
for col_name in data.columns:
    print(col_name, len(data[col_name].unique()))

id 48895
name 47906
host_id 37457
host_name 11453
neighbourhood_group 5
neighbourhood 221
latitude 19048
longitude 14718
room_type 3
price 674
minimum_nights 109
number_of_reviews 394
last_review 1765
reviews_per_month 938
calculated_host_listings_count 47
availability_365 366


In [5]:
data.isna().sum()

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

Как видите, в данных есть пропуски. Не забудьте обработать их

In [6]:
# Ваш код здесь (ノ°∀°)ノ⌒･*:.｡. .｡.:*･゜ﾟ･*☆

In [7]:
data['reviews_per_month'].fillna(0, inplace=True)
data['name'].fillna("No_name", inplace=True)
data['host_name'].fillna("No_name", inplace=True)
data['last_review'].fillna("No_reviews", inplace=True)

Разобъем данные на обучение и контроль.

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['price']), data[['price']],
                                                    test_size=0.3, random_state=241)

## Часть 1: Алгоритм kNN в задаче регрессии

<b>2.1 (1 балл)</b>
Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на [третьем семинаре](https://github.com/mmp-mmro-team/mmp_mmro_fall_2019/blob/master/lecture-notes/Sem03_knn.pdf). Не забудьте, что KNNRegressor должен уметь работать с этими функциями расстояния. Как вариант, можно реализовать метрики как [user-defined distance](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html).

In [9]:
def getComonness(array: np.array):
    """Процент каждого объекта классфикации в выборке + объект на соотв. позиции"""
    uniques, counts = np.unique(array, return_counts=True)
    return counts / np.sum(counts), uniques

def getFFunc(array: np.array):
    """Шанс встретить 2 случайных одинаковых объектов в выборке, отсортированный в словаре"""
    uniques, counts = np.unique(array, return_counts=True)
    first_func = (counts * (counts - 1)) / np.sum(counts) / (np.sum(counts) - 1)
    
    return first_func

In [10]:
from math import log
def overlap(x, z):
    """0 при совпадении, 1 иначе"""
    array = np.empty([x.__len__(), z.__len__()])
    
    for enum in range(array.__len__()):
        array[enum] = z != x[enum]
    
    return array.T

def flattened_overlap(x, z):
    
    comonness, uniques = getComonness(x)
    f_func = getFFunc(x)
    
    array = np.empty([x.__len__(), z.__len__()]).astype(float)
    
    for enum in range(array.__len__()):
        
        pos = np.where(uniques == x[enum])[0][0]
        part_sum = (np.argwhere(f_func <= f_func[pos]))
        array[enum] = (z != x[enum]).astype(float) + (z == x[enum]).astype(float) * f_func[part_sum.T[0]].sum()

    return array.T
def log_overlap(x, z):
    
    def get_pos(uniques, item):
        return np.argwhere(uniques == item)[0][0]
    
    def make_comonness_for_z():
        uniques_z, counts_z = np.unique(z, return_counts=True)
        
        comonness_line = np.empty(z.__len__())
        
        for enum, item in enumerate(z):
            comonness_line[enum] = counts_z[get_pos(uniques_z, item)]
            
        return comonness_line
        
   
    array = np.empty([x.__len__(), z.__len__()]).astype(float)
    
    uniques, comonness = np.unique(x, return_counts=True)
    comonness_line = np.log(make_comonness_for_z() + 1)
    
    for enum in range(array.__len__()):
        
        array[enum] = (z != x[enum]).astype(float) * log(comonness[get_pos(uniques, x[enum])] + 1) * comonness_line
        
    return array.T

<b> 1.1 (1.5 балла) </b>
Реализуйте класс `KNNRegressor`, который используя метод k ближайших соседей решает задачу регрессии. Для решение данной задачи, необходимо найти $N_k$ - k соседей, и после использовать значения их целевых переменных для предсказания:
\begin{align}
y = \frac{1}{k}\sum_{n \in N_k}w_n y_n,
\end{align}

где $w_n$ - вес каждого соседа. 

При этом `KNNRegressor` может работать в 2 режимах:
 - $uniform$ - ближайшие соседи учитываются с одинаковыми весами.
 - $distance$ - вес ближайших соседей зависит от расстояния
 
Сигнатуру методов при желании можно менять.

In [11]:
from typing import Union, Callable, Iterable, Optional
from sklearn.neighbors import NearestNeighbors


def make_weights(input_arr: np.array):
    all_dist = np.sum(input_arr)
    if all_dist == 0:
        return input_arr + (1/input_arr.__len__())
    return input_arr / all_dist

class KNNRegressor:
    def __init__(self, n_neighbors: int,
                 metric: Union[str, Callable],
                 mode: str = 'uniform',
                 algorithm: str = 'brute') -> None:
        """
        params:
            n_neighbors: number of neighbors
            metric: metric to use for distance computation
            mode: 'uniform' or 'distance'
            'uniform' - all points in each neighborhood are weighted equally
            'distance' - weight points by the inverse of their distance
        """
        
        self.metric = metric
        self.ne = NearestNeighbors(n_neighbors=n_neighbors, algorithm=algorithm, metric=metric, n_jobs = -1)
        
        if mode != 'uniform' and mode != 'distance':
            raise TypeError
        self.mode = mode
        self.k = n_neighbors
        
    def changeCatMetric(self, metric: Callable):
        
        if ('overlap' not in str(metric)):
            raise TypeError
            
        self.metric = metric

    def fit(self, X: np.array, y: np.array) -> None:
        """
            X: data
            y: labels
        """
        if ('overlap' in str(self.metric)):
            # здесь оно подаёт нормально, index = элемент
            if X.shape.__len__() == 1:
                self.train_set = X[:, None]
                print(X[:, None].shape, self.train_set.shape)
            else:
                self.train_set = X
            self.train_target = y
        else:
            self.ne.fit(X, y)
        
        
        
    def kneighbors(self, X: np.array, k = 0, return_distance: bool = True):
        
        if ('overlap' in str(self.metric)):
            """categorial metric area, we'd have to calculate distance for each of attributes separately
               как я решил строить входные данные для этого метода - по строкам расположены данные столбцов входных данных
               метрики будут возвращать соседей по строкам для каждого элемента по каждому из признаков
               потом мы просуммируем эти элементы"""
            
            if k == 0:
                k = self.k
                

            if X.shape.__len__() == 1:
                test = X[:, None]
            else:
                test = X
                
            # Пока что попробуем просто mae по всем элементам
            dist_by_ind = np.zeros(self.train_set.__len__() * test.__len__()).reshape(test.__len__(), -1)

            
            for index in range(self.train_set.shape[1]):
                dist_by_ind += self.metric(self.train_set.T[index], test.T[index]) ** 2
                # по идее, оно должно найти расстояния по 1 из кат. признаков
                
                
            dist_by_ind = np.sqrt(dist_by_ind)
            indexes = np.argsort(dist_by_ind, axis=1)[:, :k]
            
            if return_distance:
                distances = np.empty(indexes.shape)
                for enum, item in enumerate(indexes):
                    distances[enum] = dist_by_ind[enum, indexes[enum]]
                return distances, indexes
            
            return indexes
            
        else:
            if k == 0:
                return self.ne.kneighbors(X, return_distance=return_distance)
            return self.ne.kneighbors(X,n_neighbors=k, return_distance=return_distance)

    def predict(self, X: np.array, n_neighbors: Optional[int] = None) -> np.array:
        """
            X: data
            n_neighbors: number of neighbors
        """ 
        
        # checking for which k we have to make our neighbors
        if n_neighbors is None:
            k = self.k
        else:
            if n_neighbors > 0:
                k = n_neighbors
            else:
                k = self.k

        test_target = np.empty(X.__len__())
        
        if self.mode == 'uniform':
            nearest = self.kneighbors(X, k, return_distance=False)
            print(nearest.shape)
            for enum in range(X.__len__()):
                target = 0
                
                for numb in range(nearest[enum].__len__()):
                    target += self.train_target[nearest[enum][numb]] / k
                test_target[enum] = target

        else:
            dist, nearest = self.kneighbors(X, k, return_distance=True)

            for enum in range(X.__len__()):
                target = 0
                weights = make_weights(dist[enum])
                
                for numb in range(nearest[enum].__len__()):
                    target += self.train_target[nearest[enum, numb]] * weights[numb]

                test_target[enum] = target
                
        return test_target
        

In [12]:
knn = KNNRegressor(4, overlap, 'uniform', 'brute')

In [13]:
#X_train, X_test, y_train, y_test
train_set = np.array((X_train['name'],
                     X_train['host_name'],
                     X_train['neighbourhood_group'],
                     X_train['neighbourhood'],
                     X_train['room_type'])).T

test_set = np.array((X_test['name'],
                    X_test['host_name'], 
                    X_test['neighbourhood_group'], 
                    X_test['neighbourhood'],
                    X_test['room_type'])).T

train_target = np.array(y_train).T[0]
test_target = np.array(y_test).T[0]

In [159]:
knn.fit(train_set, train_target)

In [160]:
res = knn.predict(test_set)

(34226, 5) (14669, 5)
34226
(34226,)
[ 8784  8134  7833 16554]
(14669, 4)


In [137]:
neighbour_test = knn.kneighbors(test_set, return_distance=False)

(34226, 5) (14669, 5)
34226
(34226,)
[ 8784  8134  7833 16554]


In [201]:
neighbour_test[0,0], train_set[8784], test_set[0]

(8784, array(['Bohemian 2BR DuplexLoft in Brooklyn', 'Vida', 'Brooklyn',
        'Greenpoint', 'Entire home/apt'], dtype=object), array(['Sunny Studio Loft @ Habitat 101', 'Vida', 'Brooklyn',
        'Greenpoint', 'Entire home/apt'], dtype=object))

In [34]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [170]:
mean_squared_error(test_target, res), mean_absolute_error(test_target, res)

(54814.85601438407, 74.86263548980844)

In [171]:
knn = KNNRegressor(4, flattened_overlap, 'uniform', 'brute')

In [173]:
knn.fit(train_set, train_target)

In [174]:
res_flat = knn.predict(test_set)

(34226, 5) (14669, 5)
34226
(34226,)
[17504  7718 25529 20135]
(14669, 4)


In [186]:
knn = KNNRegressor(4, log_overlap, 'uniform', 'brute')

In [187]:
knn.fit(train_set, train_target)

In [188]:
res_log = knn.predict(test_set)

(34226, 5) (14669, 5)
34226
(34226,)
[ 6204 22157 16554  7957]
(14669, 4)


In [189]:
mean_squared_error(test_target, res_flat), mean_absolute_error(test_target, res_flat)

(50976.013762015136, 74.95340514009135)

In [191]:
mean_squared_error(test_target, res_log), mean_absolute_error(test_target, res_log)

(51577.27167836935, 73.42102392801145)

In [194]:
# я думаю, что первые 2 категориальных признака так же лишние
train_subset = np.array((X_train['neighbourhood_group'], X_train['neighbourhood'], X_train['room_type'])).T
test_subset = np.array((X_test['neighbourhood_group'], X_test['neighbourhood'], X_test['room_type'])).T

In [196]:
knn = KNNRegressor(6, flattened_overlap, 'uniform', 'brute')

In [197]:
knn.fit(train_subset, train_target)

In [198]:
res_sub = knn.predict(test_subset)

(34226, 3) (14669, 3)
34226
(34226,)
[13451 18820 33463 20846 18800 11508]
(14669, 6)


In [202]:
mean_squared_error(test_target, res_sub), mean_absolute_error(test_target, res_sub)

(46421.611967035555, 74.54311814029586)

In [17]:
"""Появилась идея рассмотреть отдельно точность для каждого признака по отдельности, тем самым понять вес каждого
рассматривать буду на flatterned_overlap"""

train_target = np.array(y_train).T[0]
test_target = np.array(y_test).T[0]

train_name, test_name = np.array(X_train['name']).T, np.array(X_test['name']).T
train_host_name, test_host_name = np.array(X_train['host_name']).T, np.array(X_test['host_name']).T
train_nhg, test_nhg = np.array(X_train['neighbourhood_group']).T, np.array(X_test["neighbourhood_group"]).T
train_nh, test_nh = np.array(X_train['neighbourhood']).T, np.array(X_test['neighbourhood']).T
train_rt, test_rt = np.array(X_train['room_type']).T, np.array(X_test['room_type']).T
train_date, test_date = np.array(X_train['last_review']).T, np.array(X_test['last_review']).T

In [101]:
knn = KNNRegressor(6, flattened_overlap, 'distance', 'brute')

In [102]:
knn.fit(train_name, train_target)


(34226, 1) (34226, 1)


In [103]:
res_name = knn.predict(test_name)

In [104]:
mean_squared_error(test_target, res_name), mean_absolute_error(test_target, res_name)

(51755.48701834181, 83.13131822928166)

In [105]:
knn.fit(train_host_name, train_target)

(34226, 1) (34226, 1)


In [106]:
res_host_name = knn.predict(test_host_name)

In [107]:
mean_squared_error(test_target, res_host_name), mean_absolute_error(test_target, res_host_name)

(51336.15712961283, 93.17247808041677)

In [108]:
knn.fit(train_nhg, train_target)

(34226, 1) (34226, 1)


In [109]:
res_nhg = knn.predict(test_nhg)

In [110]:
mean_squared_error(test_target, res_nhg), mean_absolute_error(test_target, res_nhg)

(48065.8615030942, 78.90192469379872)

In [111]:
knn.fit(train_nh, train_target)
res_nh = knn.predict(test_nh)

(34226, 1) (34226, 1)


In [112]:
mean_squared_error(test_target, res_nh), mean_absolute_error(test_target, res_nh)

(49775.301265592425, 92.98701778347296)

In [113]:
knn.fit(train_rt, train_target)
res_rt = knn.predict(test_rt)

(34226, 1) (34226, 1)


In [114]:
mean_squared_error(test_target, res_rt), mean_absolute_error(test_target, res_rt)

(45145.73344013452, 72.64044356579636)

In [115]:
knn.fit(train_date, train_target)
res_date = knn.predict(test_date)

(34226, 1) (34226, 1)


In [117]:
mean_squared_error(test_target, res_date), mean_absolute_error(test_target, res_date)

(53705.69881450402, 95.44727774998874)

In [122]:
train_combo = np.c_[train_nhg, train_rt]
test_combo = np.c_[test_nhg, test_rt]

In [124]:
knn.fit(train_combo, train_target)
res_combo = knn.predict(test_combo)

In [125]:
mean_squared_error(test_target, res_combo), mean_absolute_error(test_target, res_combo)

(44012.88233690095, 67.66209921148908)

In [ ]:
"""Интересненько...
по mae:
1) room_type (72) + neighbourhood_ghoup (78)(в принципе, логично, т.к. там мало вариантов ответа)

По mse:
опять же, они (42000) и (48000)
Эти 2 признака можно переделать через Bagofwords, они дадут не так уж и много признаков"""

## Часть 2: Категориальные признаки

<b>2.2 (1 балл)</b> Найдите все категориальные признаки в данных. Подсчитайте для каждой из метрик качество на тестовой выборке `X_test` при числе соседей $k = 10$. Качество измеряйте с помощью RMSE.

Какая функция расстояния оказалась лучшей? Почему?

In [126]:
target = np.array(data['price'])
sub_data = np.array((data['name'],
                     data['host_name'],
                     data['neighbourhood_group'],
                     data['neighbourhood'],
                     data['room_type'], 
                     data['last_review'])).T

In [128]:
train_data, test_data, train_target, test_target = train_test_split(sub_data, target,
                                                    test_size=0.3, random_state=241)

In [131]:
knn = KNNRegressor(n_neighbors=10, metric = overlap, mode = 'distance', algorithm = 'brute')

In [132]:
knn.fit(train_data, train_target)

In [134]:
res_overlap = knn.predict(test_data)

In [138]:
mean_squared_error(test_target, res_overlap), mean_absolute_error(test_target, res_overlap)

(47708.793571183625, 75.43443789988046)

In [139]:
knn.changeCatMetric(flattened_overlap)

In [147]:
res_flat = knn.predict(test_data)

In [149]:
mean_squared_error(test_target, res_flat), mean_absolute_error(test_target, res_flat)

(44504.37932704714, 72.2524153004011)

In [150]:
knn.changeCatMetric(log_overlap)

In [152]:
res_log = knn.predict(test_data)

In [154]:
mean_squared_error(test_target, res_log), mean_absolute_error(test_target, res_log)

(57100.49185801122, 86.06062500300982)

In [ ]:
# В качестве ещё 1 эксперимента, можно перевести все столбцы в формат string и прогнать на лучшем их этих 3 алгоритмов
# наилучшая метрика - flattened_overlap, т.к. она даёт гарантированно большие расстояния не совпадающим элементам

<b>2.3 (1 балл) бонус</b> Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какого удалось достичь уровня качества?

In [146]:
arr = np.arange(10)


array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='<U21')

<b>2.4 (2.5 балла)</b> Отойдем ненадолго от задачи регрессии и перейдём к задаче классификации: будем определять, являеться ли квартира дорогой $(target = 1)$ или дешевой $(target = 0)$. Будем считать дорогими квариры, цена которых выше среднего.

In [14]:
data['target'] = (data.price > data.price.mean()).astype(int)

Реализуйте счетчики, которые заменят категориальные признаки на вещественные.

А именно, для каждого категориального признака $f_j(x)$ необходимо сделать следующее:
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
\begin{align}
counts_j(c) = \sum_{i=1}^l [f_j(x_i) = c]
\end{align}
2. Число `successes` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
\begin{align}
successes_j(c) = \sum_{i=1}^l[f_j(x_i) = c][y_i = +1].
\end{align}
3. Сглаженное отношение двух предыдущих величин:
\begin{align}
p_j(c) = \frac{successes_j(c) + a}{counts_j(c) + b},
\end{align}

где $a$ и $b$ - априорные счетчики (например, a = 1, b = 2).

In [15]:
def counters(x):
    uniq, counts = np.unique(x, return_counts=True)
    
    result = np.empty(x.shape)
    
    for enum, item in enumerate(uniq):
        positions = np.argwhere(x == item)
        result[positions] = counts[enum]
    
    return result

def success_count(x, y):
    # x - cat value
    # y - target
    sub = x + y.astype(str)
    uniq, counts = np.unique(sub, return_counts=True)
    
    result = np.zeros(x.shape)
    
    for enum, item in enumerate(uniq):
        if item[-1] == '0':
            continue
        
        positions = np.argwhere(sub == item) 
        result[positions] = counts[enum]
        
        pos_neg = np.argwhere(sub == (item[:-1] + '0'))
        result[pos_neg] = counts[enum]
        
    return result

def comparison(x, y, a, b):
    count = counters(x)
    success = success_count(x, y)
    
    return (success + a) / (count + b)

In [195]:
np.unique(counters(np.array(data['neighbourhood_group']))).sum()

48895.0

In [196]:
np.unique(success_count(np.array(data['neighbourhood_group']), np.array(data['target']))).sum()

14879.0

In [194]:
np.sum(data['target'])

14879

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `successes` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанный по всей обучающей выборке. Реализуйте и такой вариант. Достаточно взять $n = 3$.

In [16]:
def kfold(n, n_folds):

    index_list = np.arange(n)

    result_list = []
    part = 0
    each_len = int(n / n_folds)

    while part < n_folds - 1:
        test_subset = np.array(index_list[part * each_len: (part + 1) * each_len])
        train_subset = np.array([x for x in index_list if x not in test_subset])
        result_list.append((train_subset, test_subset))
        part += 1

    test_subset = np.array(index_list[part * each_len:])
    train_subset = np.array([x for x in index_list if x not in test_subset])

    result_list.append((train_subset, test_subset))

    return result_list

def fold_counters(x):
    """
    params:
        x: value on categorical feature for N objects
    returns: vector of length N
    """
    
    cv = kfold(x.__len__(), 5)
    
    result = np.empty(x.__len__())
    
    for train_labels, test_labels in cv:
        count = counters(x[train_labels])
        
        extra = np.empty(test_labels.__len__())
        
        for enum, item in enumerate(x[test_labels]):
            pos = np.where(x[train_labels] == item)[0]
            if pos.__len__() == 0:
                extra[enum] = 0
            else:
                extra[enum] = count[pos[0]]
        
        result[test_labels] = extra
    
    return result

def fold_success_count(x, y):
    
    cv = kfold(x.__len__(), 5)
    
    result = np.empty(x.__len__())
        
    for train_labels, test_labels in cv:
        
        success = success_count(x[train_labels], y[train_labels])
        
        extra = np.empty(test_labels.__len__())
        
        for enum in range(test_labels.__len__()):
            cur_val = x[test_labels][enum]
            pos = np.where(x[train_labels] == cur_val)[0]
            
            if pos.__len__() == 0:
                extra[enum] = 0
            else:
                extra[enum] = success[pos[0]]
                
        result[test_labels] = extra
        
    return result

def fold_combine(x, y, a, b):
    count = fold_counters(x)
    success = fold_success_count(x, y)
    return (success + a) / (count + b)

Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

In [17]:
# Ваш код здесь
res = fold_counters(np.array(data['room_type']))

In [18]:
res2 = success_count(np.array(data['room_type']), np.array(data['target']))

In [19]:
np.unique(res2)

array([   68.,  1414., 13397.])

In [20]:
res3 = fold_success_count(np.array(data['room_type']), np.array(data['target']))

KeyboardInterrupt: 

In [ ]:
np.unique(res3)

In [ ]:
res4 = comparison(np.array(data['room_type']), np.array(data['target']), 0, 0)

In [ ]:
np.unique(res4)

In [21]:
# окей, теперь я могу провести различные эксперименты:
from sklearn.metrics import roc_auc_score

In [29]:
# для начала стоит преобразовать данные
data_test = pd.DataFrame.copy(data)
data_test.head(5)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,target
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365,0
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,1
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,No_reviews,0.00,1,365,0
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194,0
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0,0


In [30]:
data_test.drop(columns={'id', 'host_id', 'latitude', 'longitude', 'price'}, inplace = True)
data_test.head(5)

,name,host_name,neighbourhood_group,neighbourhood,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,target
0,Clean & quiet apt home by the park,John,Brooklyn,Kensington,Private room,1,9,2018-10-19,0.21,6,365,0
1,Skylit Midtown Castle,Jennifer,Manhattan,Midtown,Entire home/apt,1,45,2019-05-21,0.38,2,355,1
2,THE VILLAGE OF HARLEM....NEW YORK !,Elisabeth,Manhattan,Harlem,Private room,3,0,No_reviews,0.00,1,365,0
3,Cozy Entire Floor of Brownstone,LisaRoxanne,Brooklyn,Clinton Hill,Entire home/apt,1,270,2019-07-05,4.64,1,194,0
4,Entire Apt: Spacious Studio/Loft by central park,Laura,Manhattan,East Harlem,Entire home/apt,10,9,2018-11-19,0.10,1,0,0


In [31]:
def changeCols(df, cols, folds = False):
    
    if folds:
        desired_func = fold_combine
    else:
        desired_func = comparison
        
    for item in cols:
        df[item] = desired_func(np.array(df[item]), np.array(df['target']), 0, 0)
        
    return df

In [32]:
changeCols(data_test, ['name', 'host_name', 'neighbourhood_group', 'neighbourhood', 'room_type', 'last_review'], False)

,name,host_name,neighbourhood_group,neighbourhood,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,target
0,0.0,0.282313,0.211351,0.114286,0.063334,1,9,0.222222,0.21,6,365,0
1,1.0,0.383117,0.453257,0.699029,0.527254,1,45,0.282051,0.38,2,355,1
2,0.0,0.384615,0.453257,0.176448,0.063334,3,0,0.378731,0.00,1,365,0
3,0.0,0.000000,0.211351,0.309441,0.527254,1,270,0.274138,4.64,1,194,0
4,0.0,0.307692,0.453257,0.217547,0.527254,10,9,0.285714,0.10,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
48890,0.0,0.280000,0.211351,0.143511,0.063334,2,0,0.378731,0.00,2,9,0
48891,0.0,0.200000,0.211351,0.087627,0.063334,4,0,0.378731,0.00,2,36,0
48892,0.0,0.000000,0.453257,0.176448,0.527254,10,0,0.378731,0.00,1,27,0
48893,0.0,0.000000,0.453257,0.541369,0.058621,1,0,0.378731,0.00,6,2,0


In [33]:
data_test.head(5)

,name,host_name,neighbourhood_group,neighbourhood,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,target
0,0.0,0.282313,0.211351,0.114286,0.063334,1,9,0.222222,0.21,6,365,0
1,1.0,0.383117,0.453257,0.699029,0.527254,1,45,0.282051,0.38,2,355,1
2,0.0,0.384615,0.453257,0.176448,0.063334,3,0,0.378731,0.00,1,365,0
3,0.0,0.000000,0.211351,0.309441,0.527254,1,270,0.274138,4.64,1,194,0
4,0.0,0.307692,0.453257,0.217547,0.527254,10,9,0.285714,0.10,1,0,0


In [34]:
target_col = data_test['target']
data_test.drop(columns={'target'}, inplace = True)

In [36]:
data_test = (data_test - data_test.mean()) / data_test.std()

In [37]:
data_test.head(5)

,name,host_name,neighbourhood_group,neighbourhood,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,-0.664467,-0.083775,-0.679471,-0.925337,-1.039115,-0.293993,-0.320410,-0.837367,-0.551505,-0.034716,1.916231
1,1.519087,0.300219,1.088800,1.922185,0.961400,-0.293993,0.487660,-0.227022,-0.445075,-0.156103,1.840256
2,-0.664467,0.305928,1.088800,-0.622623,-1.039115,-0.196482,-0.522428,0.759250,-0.682979,-0.186450,1.916231
3,-0.664467,-1.159197,-0.679471,0.025008,0.961400,-0.293993,5.538099,-0.307750,2.221955,-0.186450,0.617059
4,-0.664467,0.012903,1.088800,-0.422486,0.961400,0.144805,-0.320410,-0.189654,-0.620372,-0.186450,-0.856856


In [38]:
from sklearn.neighbors import KNeighborsClassifier

In [39]:
knn = KNeighborsClassifier(n_neighbors=10, metric='cosine', weights='distance', algorithm='brute')

In [40]:
train_data, test_data, train_target, test_target = train_test_split(np.array(data_test), np.array(target_col),
                                                                    test_size=0.3, random_state=241)

In [41]:
knn.fit(train_data, train_target)

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='cosine',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='distance')

In [42]:
result = knn.predict(test_data)

In [43]:
from sklearn.metrics import accuracy_score

In [44]:
accuracy_score(test_target, result)

0.9979548708160065

In [45]:
roc_auc_score(test_target, result)

0.9975145467584173

In [48]:
from sklearn.metrics import mean_squared_error
mean_squared_error(test_target, result)

0.0020451291839934556

array([0, 1, 1, ..., 1, 0, 0])

In [313]:
# что-то мне подсказывает, что так не надо

In [332]:
train_data, test_data, train_target, test_target = train_test_split(pd.DataFrame.copy(data),
                                                                    np.array(pd.DataFrame.copy(data['target'])),
                                                                    test_size=0.3, random_state=241)

In [333]:
train_data.head(4)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,target
39319,30686870,Large bedroom,221940893,Sara,Brooklyn,Kensington,40.63628,-73.96970,Private room,60,2,12,2019-07-07,3.67,3,220,0
22796,18458323,Sun Drenched Gramercy/East Village!,68274298,Chelsea,Manhattan,Gramercy,40.73401,-73.98140,Entire home/apt,226,2,15,2017-12-18,0.58,1,0,1
26190,20886432,Gorgeous single room in Sunnyside near Manhattan,75458625,Lidia,Queens,Sunnyside,40.74722,-73.91860,Private room,78,2,20,2019-07-01,0.91,4,365,0
8161,6292334,Cozy UES Studio,7189415,Jennifer,Manhattan,Upper East Side,40.77357,-73.95193,Entire home/apt,300,1,0,No_reviews,0.00,1,0,1


In [334]:
train_data.drop(columns={'id', 'host_id'}, inplace = True)
test_data.drop(columns={'id', 'host_id'}, inplace = True)

In [335]:
changeCols(test_data, ['name', 'host_name', 'neighbourhood_group', 'neighbourhood', 'room_type', 'last_review'], False)
changeCols(train_data, ['name', 'host_name', 'neighbourhood_group', 'neighbourhood', 'room_type', 'last_review'], False)

In [336]:
train_data.drop(columns={'price', 'target'}, inplace = True)
test_data.drop(columns={'price', 'target'}, inplace=True)

In [338]:
train_target

array([0, 1, 0, ..., 0, 0, 0])

In [343]:
train_data = (train_data - train_data.mean()) / train_data.std()
test_data = (test_data - test_data.mean()) / test_data.std()

In [348]:
knn = KNeighborsClassifier(n_neighbors=10, metric='cosine', algorithm='brute', weights='distance')

In [349]:
knn.fit(np.array(train_data), train_target)

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='cosine',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='distance')

In [350]:
result_no_overlearn = knn.predict(np.array(test_data))

In [357]:
accuracy_score(test_target, result_no_overlearn)

0.9993182902720021

In [419]:
knn_cos = KNeighborsClassifier(n_neighbors=10, metric='cosine', algorithm='brute', weights='distance')
knn_euc = KNeighborsClassifier(n_neighbors=10, metric='euclidean', algorithm='brute', weights='distance')

In [426]:
train_data, test_data, train_target, test_target = train_test_split(pd.DataFrame.copy(data),
                                                                    np.array(pd.DataFrame.copy(data['target'])),
                                                                    test_size=0.3, random_state=241)

In [427]:
def transformAccordingTo(train_df, test_df, cols, folds=False):
    # окей... нужно трансформировать столбцы test так же, как столбцы train
    train_df = changeCols(train_df, cols, folds)
    for item in cols:
        
        replacement = np.empty(test_df.__len__())
        target_col = np.array(train_df[item])
        target_replacement = np.array(train_df[item+'new'])
        
        for enum, value in enumerate(test_df[item]):
            
            pos = np.argwhere(target_col == value).T[0]
            if pos.__len__() == 0:
                replacement[enum] = 0
            else:
                replacement[enum] = target_replacement[pos[0]]
                
        test_df[item] = replacement
        train_df[item] = train_df[item + 'new']
        train_df.drop(columns={item + 'new'}, inplace=True)

In [428]:
transformAccordingTo(train_data, test_data, ['name', 'host_name', 'neighbourhood_group', 'neighbourhood', 'room_type', 'last_review'])

In [429]:
train_data.drop(columns={'id', 'host_id','target', 'price'}, inplace=True)
test_data.drop(columns={'id', 'host_id', 'target', 'price'}, inplace=True)

In [430]:
train_data = (train_data - train_data.mean()) / train_data.std()
test_data = (test_data - test_data.mean()) / test_data.std()

In [431]:
knn_cos.fit(np.array(train_data), train_target)
knn_euc.fit(np.array(train_data), train_target)

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='distance')

In [435]:
accuracy_score(test_target, result_cos_no_folds), roc_auc_score(test_target, result_cos_no_folds)

(0.716885949962506, 0.5492123814181694)

In [437]:
accuracy_score(test_target, result_euc_no_folds), roc_auc_score(test_target, result_euc_no_folds)

(0.7145681368873134, 0.5447485593462442)

In [441]:
train_data, test_data, train_target, test_target = train_test_split(pd.DataFrame.copy(data),
                                                                    np.array(pd.DataFrame.copy(data['target'])),
                                                                    test_size=0.3, random_state=241)

In [442]:
transformAccordingTo(train_data,
                     test_data,
                     ['name', 'host_name', 'neighbourhood_group', 'neighbourhood', 'room_type', 'last_review'],
                     True)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:77: RuntimeWarning: invalid value encountered in true_divide


In [444]:
train_data.drop(columns={'id', 'host_id','target', 'price'}, inplace=True)
test_data.drop(columns={'id', 'host_id', 'target', 'price'}, inplace=True)
train_data = (train_data - train_data.mean()) / train_data.std()
test_data = (test_data - test_data.mean()) / test_data.std()
train_data.fillna(0, inplace = True)
test_data.fillna(0, inplace = True)

In [446]:
knn_cos.fit(np.array(train_data), train_target)
knn_euc.fit(np.array(train_data), train_target)

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='distance')

In [447]:
result_cos = knn_cos.predict(np.array(test_data))
result_euc = knn_euc.predict(np.array(test_data))

In [448]:
accuracy_score(test_target, result_cos), roc_auc_score(test_target, result_cos)

(0.80939396005181, 0.7665220124515243)

In [449]:
accuracy_score(test_target, result_euc), roc_auc_score(test_target, result_euc)

(0.8126661667461995, 0.7707152214738934)

<b>2.5 (1 балл)</b> Вернемся к задаче регрессии. Утверждается, что для задачи регрессии можно также сделать преобразование категориальных признаков в действительные числа. Для этого достаточно для каждого значения признака $f_j$ вычислить:
\begin{align}
p_j(c) = g(T_i | f_j(x_i) = c),
\end{align}

где $T_i$ - значения целевой переменной объекта $x_i$. Функция $g$ - среднее (mean) или среднеквадратичное отклонение (std).

Закодируйте категориальные признаки обоими способами и найдите значение RMSE. Используйте евклидову метрику для поиска ближайших соседей. Для какой функции $g$ значение RMSE лучше? Почему?

In [523]:
train_data, test_data, train_target, test_target = train_test_split(pd.DataFrame.copy(data.drop(columns={'target'})),
                                                                    np.array(pd.DataFrame.copy(data['price'])),
                                                                    test_size=0.3, random_state=241)

In [527]:
def Transforming_df(df, cols):
    
    for item in cols:
        
        helper = df.groupby(item, as_index=False)['price'].mean()
        
        array = np.empty(df.__len__())
        
        for enum, element in enumerate(df[item]):
            
            array[enum] = np.array(helper['price'][np.argwhere(helper[item] == element)[0]])[0]
            
        df[item+"new"] = array
    

In [528]:
def Transform_test_df(train_df, test_df, cols):
    
    Transforming_df(train_df, cols)

    for item in cols:
        
        replacement = np.empty(test_df.__len__())
        target_col = np.array(train_df[item])
        target_replacement = np.array(train_df[item+'new'])
        
        for enum, value in enumerate(test_df[item]):
            
            pos = np.argwhere(target_col == value).T[0]
            if pos.__len__() == 0:
                replacement[enum] = 0
            else:
                replacement[enum] = target_replacement[pos[0]]
                
        test_df[item] = replacement
        train_df[item] = train_df[item + 'new']
        train_df.drop(columns={item + 'new'}, inplace=True)

In [529]:
Transform_test_df(train_data, test_data, 
                 ['name', 'host_name', 'neighbourhood_group', 'neighbourhood', 'room_type', 'last_review'])

/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:61: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return bound(*args, **kwds)


In [521]:
train_data.head(4)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
39319,30686870,Large bedroom,221940893,Sara,Brooklyn,Kensington,40.63628,-73.96970,Private room,60,2,12,2019-07-07,3.67,3,220
22796,18458323,Sun Drenched Gramercy/East Village!,68274298,Chelsea,Manhattan,Gramercy,40.73401,-73.98140,Entire home/apt,226,2,15,2017-12-18,0.58,1,0
26190,20886432,Gorgeous single room in Sunnyside near Manhattan,75458625,Lidia,Queens,Sunnyside,40.74722,-73.91860,Private room,78,2,20,2019-07-01,0.91,4,365
8161,6292334,Cozy UES Studio,7189415,Jennifer,Manhattan,Upper East Side,40.77357,-73.95193,Entire home/apt,300,1,0,No_reviews,0.00,1,0


In [522]:
test_data.head(4)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
2850,1613110,Sunny Studio Loft @ Habitat 101,7503643,Vida,Brooklyn,Greenpoint,40.72570,-73.94181,Entire home/apt,129,30,8,2019-01-30,0.12,52,344
44256,34137207,"Stunning Chelsea 1BR w/ Doorman, Roofdeck, BBQ...",107434423,Blueground,Manhattan,Chelsea,40.74269,-73.99911,Entire home/apt,316,30,0,No_reviews,0.00,232,323
6636,4784867,Great location w/ Terrace!,198387,Kim,Brooklyn,Williamsburg,40.71349,-73.96290,Entire home/apt,165,5,26,2016-10-12,0.47,1,0
332,82550,Columbia Castle 2 BR,448312,Christopher,Brooklyn,Brooklyn Heights,40.69441,-73.99771,Entire home/apt,200,3,80,2019-06-30,0.85,2,106


In [504]:
train_data.drop(columns={'id', 'host_id', 'price'}, inplace=True)
test_data.drop(columns={'id', 'host_id', 'price'}, inplace=True)

In [505]:
train_data = (train_data - train_data.mean()) / train_data.std()
test_data = (test_data - test_data.mean()) / test_data.std()

In [513]:
knn_reg = KNNRegressor(n_neighbors=10, metric='euclidean', mode='uniform', algorithm = 'brute')

In [514]:
knn_reg.fit(np.array(train_data), train_target)

In [515]:
res_reg = knn.predict(np.array(test_data))

In [547]:
mean_squared_error(test_target, res_reg)

70885.96530097484

In [552]:
train_data, test_data, train_target, test_target = train_test_split(pd.DataFrame.copy(data),
                                                                    np.array(pd.DataFrame.copy(data['price'])),
                                                                    test_size=0.3, random_state=241)

In [553]:
transformAccordingTo(train_data,
                     test_data,
                     ['name', 'host_name', 'neighbourhood_group', 'neighbourhood', 'room_type', 'last_review'],
                     True)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:77: RuntimeWarning: invalid value encountered in true_divide


In [554]:
train_data.drop(columns={'id', 'host_id','target', 'price'}, inplace=True)
test_data.drop(columns={'id', 'host_id', 'target', 'price'}, inplace=True)
train_data = (train_data - train_data.mean()) / train_data.std()
test_data = (test_data - test_data.mean()) / test_data.std()
train_data.fillna(0, inplace = True)
test_data.fillna(0, inplace = True)

In [ ]:
knn = KNNRegressor(n_neighbors = k)

## Часть 3: Текстовые признаки

<b>3.1 (2 балла)</b> Перейдем от категориальным признаков к текстовым. Рассмотрим 2 способа преобразования текста в действительные числа:
- Мешок слов (Bag of Words)
- TF-IDF

[Здесь](https://scikit-learn.org/stable/modules/feature_extraction.html) вы можете прочитать про их применение в Питоне.

Сравните оба способа на задаче регресси. Какую лучше метрику использовать: евклидову или косинусную меру? Постройте графики зависимости качества решения задачи от способа преобразования, метрики и количества соседей. Мера качества - RMSE.

Объясните полученные результаты.

Перед преобразованием не забудьте уменьшить размер словаря. Например, это можно сделать за счет приведения всех слов к одному регистру и удаления [стопслов](https://en.wikipedia.org/wiki/Stop_words) (артиклей, предлогов, союзов).

In [700]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [702]:
tfidf_data = pd.DataFrame.copy(data)
tfidf_data.head(3)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,target
0,2539,Clean & quiet apt home by the park,2787,john,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365,0
1,2595,Skylit Midtown Castle,2845,jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,1
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,No_reviews,0.00,1,365,0


In [756]:
vect = TfidfVectorizer(stop_words='english')

In [757]:
X = vect.fit_transform(tfidf_data['neighbourhood_group'])

In [760]:
X.shape

(48895, 6)

In [864]:
Y = vect.fit_transform(tfidf_data['room_type'])

In [865]:
Y.shape

(48895, 6)

In [867]:
vect.get_feature_names()

['apt', 'entire', 'home', 'private', 'room', 'shared']

In [765]:
Z = vect.fit_transform(tfidf_data['host_name'])

In [862]:
Z.shape

(48895, 10018)

In [863]:
np.unique(Z.toarray().sum(axis=1)).__len__()

1238

In [769]:
K = vect.fit_transform(tfidf_data['name'])

In [855]:
K.shape

(48895, 7865)

In [861]:
np.unique(K.toarray().sum(axis=1)).__len__()

44101

In [772]:
H = vect.fit_transform(tfidf_data['neighbourhood'])

In [773]:
H.shape

(48895, 240)

In [775]:
J = vect.fit_transform(tfidf_data['last_review'])

In [776]:
J.shape

(48895, 41)

In [664]:
df = pd.DataFrame.copy(data)

In [531]:
# Преобразование входных данных: удалим "," "." ":" "/" "\", двойные пробелы и т.д.
text_transform = pd.DataFrame.copy(data)

In [545]:
first_col = np.array(text_transform['name'])
first_col[643]

'Williamsburg  - Quiet and Comfy stay'

In [536]:
def mass_replace(string, deletions):
    
    for item in deletions:
        string = string.replace(item, "")
        
    return string

In [571]:
for enum, item in enumerate(first_col):
    first_col[enum] = item.lower().replace("  ", " ")

for enum, item in enumerate(first_col):
    first_col[enum] = item.replace("!", "")
    
for enum, item in enumerate(first_col):
    first_col[enum] = item.replace(",", "")

In [572]:
uniq_dict = {}

for item in first_col:
    line = item.split(" ")
    
    for part in line:
        if part in uniq_dict:
            uniq_dict[part] += 1
        else:
            uniq_dict[part] = 1

In [573]:
sorted_dict = [(k, uniq_dict[k]) for k in sorted(uniq_dict, key=uniq_dict.get, reverse=True)]
sorted_dict

[('in', 16735),
 ('room', 9794),
 ('bedroom', 7502),
 ('private', 7091),
 ('apartment', 6475),
 ('cozy', 4954),
 ('brooklyn', 3938),
 ('apt', 3927),
 ('studio', 3909),
 ('the', 3869),
 ('to', 3827),
 ('spacious', 3704),
 ('1', 3359),
 ('manhattan', 3246),
 ('with', 3091),
 ('2', 3090),
 ('of', 2993),
 ('east', 2989),
 ('park', 2965),
 ('and', 2869),
 ('sunny', 2867),
 ('&', 2818),
 ('williamsburg', 2570),
 ('beautiful', 2467),
 ('near', 2289),
 ('-', 2271),
 ('village', 2204),
 ('nyc', 2054),
 ('heart', 2044),
 ('large', 2037),
 ('loft', 1936),
 ('a', 1916),
 ('modern', 1769),
 ('home', 1750),
 ('central', 1738),
 ('from', 1699),
 ('bright', 1676),
 ('luxury', 1648),
 ('west', 1541),
 ('new', 1510),
 ('location', 1500),
 ('1br', 1491),
 ('bed', 1445),
 ('charming', 1371),
 ('side', 1354),
 ('upper', 1318),
 ('for', 1308),
 ('bushwick', 1291),
 ('midtown', 1208),
 ('one', 1192),
 ('quiet', 1191),
 ('w/', 1174),
 ('brownstone', 1156),
 ('great', 1135),
 ('br', 1106),
 ('on', 1070),
 ('cl

In [627]:
name_cat = pd.DataFrame(data['name'], index=np.arange(data.__len__()), columns=['name'])

In [628]:
name_cat['bedroom'] = 0
name_cat['apartment'] = 0
name_cat['brooklyn'] = 0
name_cat['manhattan'] = 0
name_cat['queens'] = 0
name_cat['1br'] = 0
name_cat['2br'] = 0
name_cat['3br'] = 0
name_cat['east'] = 0
name_cat['west'] = 0
name_cat['central'] = 0
# 😡😤

In [629]:
for enum, item in enumerate(first_col):
    line = item.split(" ")
    
    def modify(args, col):
        for item in args:
            if item in line:
                name_cat[col][enum] = 1
            
    modify(["room", "bedroom", "bed"], "bedroom")
    modify(["apartment", "apt", "studio"], "apartment")
    modify(["brooklyn", "greenpoint", "williansburg", "bushwick"], "brooklyn")
    modify(["manhattan", "kitchen", "soho", "village", "greenwich", "harlem"], "manhattan")
    modify(["queens", "queen", "astoria"], "queens")
    modify(["1", "one", "1bd", "1br", "1-bedroom"], "1br")
    modify(["2", "2br", "2bd", "2-bedroom", "two"], "2br")
    modify(["3", "3br"], "3br")
    modify(["east"], "east")
    modify(["west"], "west")
    modify(["central"], "central")

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [634]:
name_cat.drop(columns={"name"}, inplace=True)

In [636]:
name_cat.head(3)

,bedroom,apartment,brooklyn,manhattan,queens,1br,2br,3br,east,west,central
0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0


In [640]:
for col_name in name_cat:
    print(col_name, name_cat[col_name].sum())

bedroom 18111
apartment 13749
brooklyn 5590
manhattan 7387
queens 1293
1br 6414
2br 4202
3br 1194
east 2978
west 1540
central 1735


In [643]:
second_col = np.array(data['host_name'])

In [644]:
for enum, item in enumerate(second_col):
    second_col[enum] = item.lower()

In [695]:
uniq_dict_host = {}
uniq_list_host = []

for item in second_col:
    line = item.split(" ")
    if item not in uniq_list_host:
        uniq_list_host.append(item)
    if line.__len__() > 2:
        print(line)
    
    for part in line:
        if part in uniq_dict_host:
            uniq_dict_host[part] += 1
        else:
            uniq_dict_host[part] = 1

['allen', '&', 'irina']
['adam', 'and', 'charity']
['claude', '&', 'sophie']
['ssameer', 'or', 'trip']
['ssameer', 'or', 'trip']
['jt', 'and', 'tiziana']
['tye', 'and', 'etienne']
['david', 'and', 'rick']
['sean', '&', 'lynette']
['ssameer', 'or', 'trip']
['fernando', 'and', 'lenin']
['ricardo', '&', 'ashlie']
['dennis', '&', 'naoko']
['kristian', '&', 'di']
['mary', 'and', 'geoff']
['g', '&', 's']
['the', 'box', 'house', 'hotel']
['the', 'box', 'house', 'hotel']
['the', 'box', 'house', 'hotel']
['the', 'box', 'house', 'hotel']
['the', 'box', 'house', 'hotel']
['liz', 'and', 'melissa']
['lee', 'and', 'tara']
['the', 'box', 'house', 'hotel']
['dee,', 'dre', '&', 'mama', 'shelley']
['gladys', '&', 'bob']
['lennny', '&', 'megan']
['jt', 'and', 'tiziana']
['jt', 'and', 'tiziana']
['linda', '&', 'chris']
['ali', '&', 'sweetpea']
['christian', '&', 'carla']
['andrew', '&', 'markus']
['welcome', 'to', 'my', 'place']
['zora', '&', 'chris']
['keenan', '&', 'emily']
['boubacar', 'and', 'pattee']

['kameryn', '&', 'rob']
['robin', '&', 'bob']
['gurpreet', '', 'singh']
['gurpreet', '', 'singh']
['blue', '&', 'elli']
['genevieve', '&', 'brett']
['beatriz', '&', 'will']
['dan', 'and', 'lily']
['seema', 'n', 'james']
['agathe', 'et', 'francois']
['tom', '&', 'nic']
['(ari)', 'henry', 'lee']
['seun', 'and', 'marie']
['arianna', '&', 'z']
['george', '-', 'francis']
['linda', '&', 'didier']
['andres', 'and', 'salina']
['brooklyn&', '', '', 'breakfast', '', '', '', '-len-']
['alicia', 'and', 'zach']
['tzooli', 'and', 'ay']
['chad', '&', 'david']
['jovan', '&', 'zaga']
['amelie', '&', 'pascal']
['yasu', '&', 'akiko']
['brooklyn&', '', '', 'breakfast', '', '', '', '-len-']
['andrew', 'and', 'talia']
['andrew', 'and', 'talia']
['basil', 'and', 'jacqueline']
['basil', 'and', 'jacqueline']
['brooklyn&', '', '', 'breakfast', '', '', '', '-len-']
['sam', 'and', 'roxanna']
['alec', '&', 'laura']
['crystal', '&', 'allan']
['frank', 'and', 'anna']
['anamaria', '&', 'ricardo']
['eric', 'and', 'aoi

['yosef', 'and', 'chava']
['eddie', '&', 'lois']
['tony', '&', 'grace']
['tony', '&', 'grace']
['brian', '&', 'jonathan']
['jovanni', '&', 'natasha']
['endless', 'resort', 'options']
['tim', '&', 'rachel']
['tony', '&', 'grace']
['sarah', '&', 'jc']
['caro', '&', 'allen']
['jeremy', '&', 'laura']
['gennady', '+', 'laura']
['jeremy', '&', 'laura']
['caroline', '&', 'georg']
['lara', '&', 'david']
['hurriyet', 'aydın', '/']
['chad', '&', 'noel']
['jennifer', '&', 'inam']
['lauren', '&', 'anthony']
['jeanne', '&', 'sara']
['diane', 'and', 'ward']
['dave', '+', 'suzanne']
['eleanor', '&', 'kameon']
['jeremy', '&', 'laura']
['jessica', 'and', 'danny']
['megan', '&', 'matt']
['yoseph', '&', 'malky']
['jeremy', '&', 'laura']
['sophia', 'and', 'shannon']
['jane', '+', 'abe']
['kim', '&', 'rupert']
['jeremy', '&', 'laura']
['will', '&', 'kim']
['edna', '+', 'isaiah']
['jen', 'and', 'lev']
['mark', '&', 'will']
['caroline', '&', 'georg']
['giedre', '&', 'andre']
['davis', '&', 'coco']
['shani', 

['josh', '&', 'madeleine']
['ali', '&', 'guillaume']
['marc', '&', 'youna']
['adesh', '&', 'viviana']
['pablo', '&', 'anastasiya']
['cj', '&', 'raissa']
['john', '&', 'christine']
['lilia', 'and', 'mateo']
['ricardo', '&', 'ashlie']
['colin', 'and', 'bryn']
['isaac', 'and', 'eunice']
['ruth', '&', 'c']
['yado', '&', 'vane']
['debbie', '&', 'hilel']
['hannes', '&', 'will']
['pedro', 'y', 'martha']
['german', 'and', 'wendy']
['andreas', '&', 'bella']
['andreas', '&', 'bella']
['monique', 'and', 'colin']
['lyric', 'at', '70', 'pine']
['erika', '&', 'brett']
['lyric', 'at', '70', 'pine']
['lyric', 'at', '70', 'pine']
['akem', '&', 'terry']
['anh', '&', 'david']
['lillie', '&', 'donald']
['jeffrey', '&', 'casandra']
['missy', '&', 'julien']
['chris', '&', 'grace']
['sarah', '&', 'bruce']
['the', 'bowery', 'house']
['william', 'and', 'julian']
['hyesuk', '&', 'alex']
['the', 'bowery', 'house']
['the', 'bowery', 'house']
['hyesuk', '&', 'alex']
['thom', '&', 'moses']
['hyesuk', '&', 'alex']
[

['yao', '&', 'rain']
['vicky', 'and', 'nektarios']
['isa', '&', 'bruno']
['isa', '&', 'bruno']
['isa', '&', 'bruno']
['cassa', 'times', 'square', 'hotel']
['cassa', 'times', 'square', 'hotel']
['cassa', 'times', 'square', 'hotel']
['cassa', 'times', 'square', 'hotel']
['cassa', 'times', 'square', 'hotel']
['ashley', 'at', 'bedly']
['ashley', 'at', 'bedly']
['ashley', 'at', 'bedly']
['sandra', 'and', 'katharina']
['mark', '&', 'olivia']
['ashley', 'at', 'bedly']
['raine', '&', 'andrew']
['sara', 'and', 'ronald']
['coleen,', 'felipe', 'and', 'victoria']
['minami', '&', 'takuya']
['renee', '&', 'roscoe']
['lia', 'and', 'patrick']
['kevin', 'and', 'karen']
['lee', 'and', 'teri']
['tim', '&', 'cecilia']
['rabi', '&', 'soukaina']
['nikoulas', '&', 'julie']
['fabio', '&', 'angie']
['peter', '&', 'giulia']
['fanny', '+', 'matthew']
['dmitriy', '&', 'amy']
['david', 'and', 'annette']
['david', 'and', 'annette']
['jess', '&', 'ana']
['hugo', '&', 'amelia']
['jenn', 'and', 'mike']
['june', 'and',

In [698]:
uniq_list_host

['john',
 'jennifer',
 'elisabeth',
 'lisaroxanne',
 'laura',
 'chris',
 'garon',
 'shunichi',
 'maryellen',
 'ben',
 'lena',
 'kate',
 'laurie',
 'claudio',
 'alina',
 'allen & irina',
 'jane',
 'doti',
 'adam and charity',
 'sing',
 'chaya',
 'lisel',
 'nathalie',
 'gregory',
 'claude & sophie',
 'tommi',
 'shon',
 'dana',
 'ssameer or trip',
 'teri',
 'andrea',
 'angela',
 'vt',
 'tyrome',
 'harriet',
 'edward',
 'abdul',
 'yolande',
 'cyn',
 'earl',
 'rana',
 'orestes',
 'adreinne',
 'alexander',
 'jt and tiziana',
 'joya',
 'james',
 'jeanne',
 'francesca',
 'joanna',
 'bianca',
 'luiz',
 'ted',
 'cristina',
 'petra',
 'd',
 'dimitri',
 'patricia',
 'mark',
 'sara',
 'reka',
 'daniel',
 'casey',
 'robin',
 'anna',
 'enzo',
 'tye and etienne',
 'george',
 'josh',
 'victoria',
 'justin',
 'blaise',
 'david and rick',
 'lulú',
 'sybilla',
 'jolynn',
 'gaia',
 'ana',
 'maggie',
 'starlee',
 'pas',
 'augustin',
 'sean & lynette',
 'erica',
 'tracy',
 'christiana',
 'elliott',
 'olan',


In [681]:
sorted_dict_host = [(k, uniq_dict_host[k]) for k in sorted(uniq_dict_host, key=uniq_dict_host.get, reverse=True)]
sorted_dict_host

[('&', 1054),
 ('and', 619),
 ('michael', 457),
 ('david', 449),
 ('sonder', 423),
 ('john', 337),
 ('alex', 330),
 ('(nyc)', 327),
 ('laura', 293),
 ('maria', 244),
 ('daniel', 242),
 ('sarah', 240),
 ('blueground', 232),
 ('jessica', 218),
 ('anna', 217),
 ('andrew', 210),
 ('mike', 208),
 ('chris', 204),
 ('james', 182),
 ('emily', 181),
 ('melissa', 165),
 ('jennifer', 163),
 ('michelle', 159),
 ('rachel', 158),
 ('ben', 156),
 ('jason', 155),
 ('mark', 154),
 ('jonathan', 153),
 ('brian', 150),
 ('amy', 149),
 ('adam', 148),
 ('peter', 148),
 ('jeremy', 147),
 ('kara', 146),
 ('anthony', 141),
 ('kevin', 141),
 ('eric', 139),
 ('nick', 136),
 ('andrea', 135),
 ('lauren', 135),
 ('joseph', 132),
 ('nina', 131),
 ('hotel', 128),
 ('ken', 128),
 ('sam', 128),
 ('elizabeth', 127),
 ('lisa', 126),
 ('robert', 126),
 ('paul', 126),
 ('stephanie', 126),
 ('nicole', 125),
 ('matt', 125),
 ('ryan', 120),
 ('ashley', 117),
 ('max', 115),
 ('matthew', 114),
 ('joe', 114),
 ('sara', 111),
 ('

<b>3.2 (1 балл)</b> Используя все доступные признаки, решите задачу регрессии. Для категориальных и текстовых признаков выберите лучшие преобразования. Повлияло ли добавление количественного признака на метрику качества?

In [581]:
trying = data.groupby("host_id", as_index=False)['price', 'calculated_host_listings_count'].mean()

In [596]:
trying.sort_values('price').head(10)

,host_id,price,calculated_host_listings_count
11495,10132166,0.0,1.0
13414,13709292,0.0,1.0
33762,197169969,10.0,1.0
20370,33511962,10.0,1.0
23429,47336995,10.0,1.0
24685,52777892,10.0,1.0
1609,647528,10.0,1.0
34080,205820814,10.0,1.0
25797,62685070,10.0,1.0
12691,11967922,10.0,1.0


In [624]:
data.loc[data['host_id'] == 10132166] # я нашел, реально стоит 6300 рублей\день

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,target
25778,20624541,Modern apartment in the heart of Williamsburg,10132166,Aymeric,Brooklyn,Williamsburg,40.70838,-73.94645,Entire home/apt,0,5,3,2018-01-02,0.15,1,73,0


In [689]:
data.loc[data['host_name'] == "the box house hotel"]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,target
321,77765,Superior @ Box House,417504,the box house hotel,Brooklyn,Greenpoint,40.73749,-73.95292,Private room,179,3,36,2019-07-01,0.36,28,79,1
327,80684,Duplex w/ Terrace @ Box House Hotel,417504,the box house hotel,Brooklyn,Greenpoint,40.73776,-73.95327,Private room,349,3,8,2016-03-27,0.09,28,60,1
328,80700,Loft w/ Terrace @ Box House Hotel,417504,the box house hotel,Brooklyn,Greenpoint,40.73738,-73.95482,Private room,349,3,7,2019-05-24,0.07,28,60,1
330,81739,Loft w/ Terrace @ Box House Hotel,417504,the box house hotel,Brooklyn,Greenpoint,40.73842,-73.95312,Private room,249,3,2,2011-05-12,0.02,28,60,1
339,84010,Superior @ Box House,417504,the box house hotel,Brooklyn,Greenpoint,40.73813,-73.95394,Private room,179,3,13,2019-06-27,0.14,28,81,1
365,103311,2 BR w/ Terrace @ Box House Hotel,417504,the box house hotel,Brooklyn,Greenpoint,40.73861,-73.95485,Private room,599,3,9,2018-05-19,0.09,28,60,1
652,248865,Loft Suite @ Box House Hotel,417504,the box house hotel,Brooklyn,Greenpoint,40.73787,-73.95385,Entire home/apt,199,3,34,2019-04-22,0.37,28,60,1
668,253466,Loft Suite @ The Box House Hotel,417504,the box house hotel,Brooklyn,Greenpoint,40.73693,-73.95284,Entire home/apt,199,3,33,2019-06-24,0.47,28,60,1
669,253471,Loft Suite @ The Box House Hotel,417504,the box house hotel,Brooklyn,Greenpoint,40.73641,-73.95330,Entire home/apt,199,3,24,2018-11-06,0.32,28,84,1
670,253475,Loft Suite @ The Box House Hotel,417504,the box house hotel,Brooklyn,Greenpoint,40.73794,-73.95254,Entire home/apt,199,3,59,2019-06-24,0.66,28,60,1


## Часть 4: Выводы


In [19]:
# Ваши выводы здесь (ノ°∀°)ノ⌒･*:.｡. .｡.:*･゜ﾟ･*☆

In [40]:
arr_test = np.arange(10).reshape(2, 5
arr_test[0, 3] = 0

In [41]:
target = arr_test / 0

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [42]:
target[target == np.inf] = 0

In [46]:
target[np.isnan(target)] = 0

In [47]:
target

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [844]:
x = np.array([1, 2, 2, 1, 5, 1]).astype(str)
z = np.array([1, 2, 3]).astype(str)
print(log_overlap(x, z))

[[0.         0.76150001 0.76150001 0.         0.48045301 0.        ]
 [0.96090603 0.         0.         0.96090603 0.48045301 0.96090603]
 [0.96090603 0.76150001 0.76150001 0.96090603 0.48045301 0.96090603]]


In [868]:
from sklearn.feature_extraction.text import CountVectorizer

In [869]:
vectoriser = CountVectorizer(stop_words='english')

In [870]:
Y = vectoriser.fit_transform(data['name'])

In [881]:
vectoriser.get_feature_names().__len__()

7865

In [882]:
stop_list = ['herself', 'other', 'has', 'twenty', 'seeming', 'last', 'been', 'many', 'namely', 'per', 'you', 'whenever', 'in', 'hence', 'often', 'sincere', 'everywhere', 'yours', 'ours', 'find', 'done', 'somehow', 'formerly', 'alone', 'whose', 'system', 'seems', 'together', 'over', 'himself', 'elsewhere', 'already', 'call', 'ltd', 'sixty', 'mostly', 'still', 'or', 'yourself', 'thence', 'nobody', 'themselves', 'thick', 'de', 'have', 'no', 'upon', 'these', 'anyhow', 'un', 'couldnt', 'thin', 'detail', 'be', 'much', 'while', 'fifteen', 'co', 'from', 'now', 'herein', 'besides', 'can', 'might', 'itself', 'latter', 'could', 'wherever', 'none', 'anywhere', 'whereafter', 'else', 'thereupon', 'had', 'mine', 'anyone', 'take', 'nine', 'something', 'neither', 'nothing', 'twelve', 'again', 'side', 'are', 'beyond', 'than', 'back', 'hereafter', 'less', 'do', 'against', 'beside', 'your', 'me', 'three', 'were', 'forty', 'meanwhile', 'not', 'whom', 'etc', 'fifty', 'whole', 'several', 'afterwards', 'almost', 'during', 'con', 'thus', 'for', 'move', 'bottom', 'everyone', 'though', 'becoming', 'who', 'becomes', 'until', 'mill', 'bill', 'above', 'between', 'whereby', 'was', 'would', 'they', 'us', 'nor', 'never', 'there', 'cannot', 'may', 'former', 'what', 'please', 'top', 'further', 'empty', 'part', 'which', 'up', 'describe', 'whereupon', 'but', 'via', 'very', 'at', 'ever', 'therefore', 'with', 'latterly', 'within', 'full', 'a', 'get', 'along', 'i', 'next', 'either', 'hers', 'of', 'found', 'whether', 'should', 'two', 'although', 'down', 'to', 'front', 'eg', 'yet', 'thru', 'anything', 'on', 'since', 'perhaps', 'such', 'hereby', 'under', 'where', 'somewhere', 'any', 'one', 'behind', 'being', 'six', 'give', 'thereafter', 'every', 'my', 'five', 'hasnt', 'without', 'amongst', 'four', 'first', 'is', 'our', 'he', 'here', 'someone', 'it', 'by', 'when', 'became', 'more', 'most', 'own', 'third', 'others', 'keep', 'why', 'nevertheless', 'must', 'will', 'across', 'as', 'eight', 'after', 'whence', 'amoungst', 'sometimes', 'then', 'into', 'even', 'so', 'also', 'if', 'whereas', 'thereby', 'her', 'among', 're', 'throughout', 'seem', 'hundred', 'amount', 'therein', 'cant', 'fill', 'serious', 'and', 'that', 'anyway', 'well', 'myself', 'everything', 'ourselves', 'toward', 'indeed', 'about', 'we', 'made', 'see', 'noone', 'how', 'except', 'inc', 'his', 'otherwise', 'sometime', 'go', 'onto', 'those', 'yourselves', 'off', 'eleven', 'them', 'both', 'whither', 'ten', 'him', 'beforehand', 'ie', 'out', 'few', 'same', 'all', 'she', 'whatever', 'before', 'due', 'its', 'their', 'towards', 'am', 'always', 'the', 'seemed', 'only', 'because', 'cry', 'enough', 'fire', 'nowhere', 'show', 'an', 'least', 'too', 'once', 'put', 'interest', 'through', 'below', 'another', 'each', 'wherein', 'some', 'become', 'name', 'whoever', 'around', 'rather', 'this', 'hereupon', 'moreover', 'however']

In [923]:
#  Попробуем преборазовать

transf_array = np.array(data['name'])

In [924]:
def mass_replace(string, deletions):
    
    for item in deletions:
        string = string.replace(item, "")
        
    return string

In [925]:
for enum, item in enumerate(transf_array):
    transf_array[enum] = item.lower()
    transf_array[enum] = transf_array[enum].replace("  ", " ")
    transf_array[enum] = transf_array[enum].replace("/", " ")
    transf_array[enum] = transf_array[enum].replace("-", " ")
    transf_array[enum] = transf_array[enum].replace("...", " ")

for enum in range(transf_array.__len__()):
    
    transf_array[enum] = mass_replace(transf_array[enum], ["!", ",", "&", ".", "'", ":", "*"])
    
    

In [895]:
stop_list.append("")

In [926]:
uniq_dict_host = {}
first_col = []

for item in transf_array:
    line = item.split(" ")
    line_kept = [x for x in line if x not in stop_list]
    first_col.append(line_kept)
    
    for part in line_kept:
        line_cur = []
        if part not in line_cur:
            line_cur.append(part)
            if part in uniq_dict_host:
                uniq_dict_host[part] += 1
            else:
                uniq_dict_host[part] = 1

In [927]:
sorted_dict = [(k, uniq_dict_host[k]) for k in sorted(uniq_dict_host, key=uniq_dict_host.get, reverse=True)]
sorted_dict

[('room', 10098),
 ('bedroom', 8150),
 ('private', 7312),
 ('apartment', 6734),
 ('cozy', 5018),
 ('apt', 4668),
 ('studio', 4074),
 ('brooklyn', 4069),
 ('1', 3891),
 ('spacious', 3755),
 ('2', 3539),
 ('manhattan', 3441),
 ('park', 3106),
 ('east', 3073),
 ('sunny', 2904),
 ('williamsburg', 2701),
 ('beautiful', 2491),
 ('near', 2325),
 ('village', 2307),
 ('nyc', 2203),
 ('w', 2105),
 ('large', 2065),
 ('loft', 2064),
 ('heart', 2062),
 ('bed', 2008),
 ('home', 1854),
 ('modern', 1803),
 ('central', 1792),
 ('bright', 1701),
 ('luxury', 1690),
 ('west', 1594),
 ('location', 1579),
 ('new', 1554),
 ('1br', 1540),
 ('bushwick', 1409),
 ('charming', 1378),
 ('upper', 1349),
 ('br', 1279),
 ('midtown', 1254),
 ('quiet', 1220),
 ('brownstone', 1183),
 ('great', 1163),
 ('harlem', 1133),
 ('3', 1124),
 ('clean', 1118),
 ('square', 1071),
 ('close', 1043),
 ('+', 1029),
 ('bath', 985),
 ('garden', 983),
 ('subway', 974),
 ('|', 963),
 ('huge', 960),
 ('heights', 934),
 ('times', 868),
 ('d

In [928]:
# building tf-idf metric
words_list = []
numbers_list = []

for item in sorted_dict:
    words_list.append(item[0])
    numbers_list.append(item[1])

In [930]:
# counting idf for each word:
idf = np.empty(words_list.__len__())
for enum in range(words_list.__len__()):
    idf[enum] = log(48995 / numbers_list[enum])

In [952]:
words = np.array(words_list)
numbers = np.array(numbers_list)

In [956]:
tf_idf = np.zeros(first_col.__len__())

for enum, item in enumerate(first_col):
    uniq, count = np.unique(np.array(item), return_counts=True)
    tf = count / np.sum(count)
    for enumer, word in enumerate(uniq):
        pos = np.argwhere(words == word)[0][0]
        tf_idf[enum] += tf[enumer] * idf[pos]

In [957]:
np.max(idf)

10.79947353106992

In [960]:
np.max(tf_idf), np.min(tf_idf)

(10.799473531069921, 0.0)